In [16]:
import numpy as np
import os.path
import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, accuracy_score, classification_report
from models.mlp import MLP
from torch.utils.data import Dataset, DataLoader

Sklearn train and test loaders

In [38]:
fractional_cover = pd.read_csv("datasets/fractional_cover.csv")
fractional_cover["tree_type"] = fractional_cover["plot_ID"].apply(lambda x: ''.join([char for char in x.split("_")[1] if char.isalpha()]))

X = pd.read_csv("datasets/1000_wavelength_10features_pca.csv").to_numpy()
y_intermediate_points = pd.read_csv("datasets/complete_fc.csv")
y_intermediate_points = y_intermediate_points[["vasc", "nonvasc", "lichen", "intactlitt", "decomplitt"]].to_numpy() * 100
y_plot_ids = fractional_cover["plot_ID"].to_numpy().repeat(15)
y_plot_ids = np.delete(y_plot_ids, np.where(y_plot_ids == "HY_PINE5")[0][0])
y_tree_types = fractional_cover["tree_type"].to_numpy().repeat(15)
y_tree_types = np.delete(y_tree_types, np.where(y_tree_types == "PINE")[0][0])

Predict using SVM.

In [39]:
# SVM code here
clf = SVC(kernel='linear')
X_train, X_test, y_train, y_test = train_test_split(X, y_plot_ids, test_size=0.2, random_state=42)
clf = clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.6149


In [43]:
clf = SVC(kernel='linear')
X_train, X_test, y_train, y_test = train_test_split(X, y_tree_types, test_size=0.2, random_state=42)
clf = clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.6954


Predict fractional cover measurements at individual locations using MLP.

In [66]:
mlp = MLPRegressor(hidden_layer_sizes=(100,), activation="tanh", solver="adam", batch_size=256, learning_rate="constant", learning_rate_init=0.001, max_iter=10000)

X_train, X_test, y_train, y_test = train_test_split(X, y_intermediate_points, test_size=0.1)

mlp.fit(X_train, y_train)
targets = mlp.predict(X_test)

print(f"Errors: {np.abs(np.round(targets - y_test, decimals=1))}")
print(f"MSE loss: {mean_squared_error(y_test, targets)}")
print(f"MAE loss: {mean_absolute_error(y_test, targets)}")

Errors: [[10.1 10.   0.1  4.4  3. ]
 [11.7  4.1  0.8  1.1 12.1]
 [15.5 19.4  0.4  7.2  1.1]
 [ 9.1  8.   0.1 15.   1.9]
 [49.2 23.6  0.8  7.7  1.7]
 [ 0.1  1.1  0.4  0.7  3.1]
 [ 1.9  5.7  0.1 10.2  0. ]
 [35.4 21.7  0.4 12.   0.7]
 [ 6.8  1.   0.3  2.7 13.4]
 [ 4.5  6.9  1.  18.9  5.8]
 [69.   5.   1.1 50.7  9.1]
 [14.  18.1  0.2 27.7  1.8]
 [17.3 15.7  0.3 32.2  4.5]
 [11.8  4.3  1.  13.4  3.7]
 [17.3  2.5  0.2 15.3  5.2]
 [ 9.6  3.3  0.8 25.7 16.1]
 [20.4 17.6  0.1 33.6  1.7]
 [12.6  0.9  0.4  9.6 26.7]
 [ 1.9  3.5  0.7  9.2  3. ]
 [14.1  4.1  0.2 21.9  6.2]
 [14.4  4.2  0.2  9.   1.3]
 [ 5.2  2.3  0.2  3.4  1.4]
 [ 8.8  8.6  0.3  1.2  1.9]
 [14.2  2.5  0.3  2.3 17.2]
 [ 4.1 36.7  0.3 25.9  1.8]
 [17.7  4.5  1.   6.7  0.1]
 [ 3.4 10.3  0.7 15.2  2.4]
 [22.3  1.4  0.2 18.5  9. ]
 [10.7  4.4  0.5 13.7  2.2]
 [16.9 20.4  0.4  6.9  2.2]
 [16.   9.6  0.1 11.7  1. ]
 [ 5.4  7.1  0.1 17.8  1.5]
 [31.9 37.7  0.8  8.4  1.4]
 [14.8  6.3  0.  19.9  1.2]
 [12.2  2.8  0.7 12.   0.8]
 [25.8  5.7 

Predict plots using MLP.

In [97]:
mlp = MLPClassifier(hidden_layer_sizes=(100,), activation="tanh", solver="adam", batch_size=256, learning_rate="constant", learning_rate_init=0.001, max_iter=10000)

X_train, X_test, y_train, y_test = train_test_split(X, y_plot_ids, test_size=0.1)

mlp.fit(X_train, y_train)
targets = mlp.predict(X_test)

print(f"Accuracy of predictions: {accuracy_score(y_test, targets)}")

Accuracy of predictions: 0.5632183908045977


Predict tree types using MLP.

In [104]:
mlp = MLPClassifier(hidden_layer_sizes=(100,), activation="tanh", solver="adam", batch_size=256, learning_rate="constant", learning_rate_init=0.001, max_iter=10000)

X_train, X_test, y_train, y_test = train_test_split(X, y_tree_types, test_size=0.1)

mlp.fit(X_train, y_train)
targets = mlp.predict(X_test)

print(f"Accuracy of predictions: {accuracy_score(y_test, targets)}")


Accuracy of predictions: 0.6206896551724138
